# Change table data by using compaction
<!--
  ~ Licensed to the Apache Software Foundation (ASF) under one
  ~ or more contributor license agreements.  See the NOTICE file
  ~ distributed with this work for additional information
  ~ regarding copyright ownership.  The ASF licenses this file
  ~ to you under the Apache License, Version 2.0 (the
  ~ "License"); you may not use this file except in compliance
  ~ with the License.  You may obtain a copy of the License at
  ~
  ~   http://www.apache.org/licenses/LICENSE-2.0
  ~
  ~ Unless required by applicable law or agreed to in writing,
  ~ software distributed under the License is distributed on an
  ~ "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
  ~ KIND, either express or implied.  See the License for the
  ~ specific language governing permissions and limitations
  ~ under the License.
  -->

Through compaction, whether manual or running automatically, you can change the schema of a table, filter data out, and apply data transformations - whether that is the entire table, or just data of a particular age.

This tutorial demonstrates how to work with [compaction](https://druid.apache.org/docs/latest/data-management/compaction) to apply different dimension schemes and to apply transformations.

In this tutorial you perform the following tasks:

- Create a table using batch ingestion.
- Run a compaction task to remove dimensions for a particular time period in the data.
- Run a compaction task to apply an expression to the data in a table.
- Run a compaction task to remove all data that matches a particular criteria.

## Prerequisites

This tutorial works with Druid 30.0.0 or later.

Launch this tutorial and all prerequisites using the `druid-jupyter` profile of the Docker Compose file for Jupyter-based Druid tutorials. For more information, see the Learn Druid repository [readme](https://github.com/implydata/learn-druid).
   

## Initialization

The following cells set up the notebook and learning environment ready for use.

### Set up a connection to Apache Druid

Run the next cell to set up the Druid Python client's connection to Apache Druid.

If successful, the Druid version number will be shown in the output.

In [ ]:
import druidapi
import os

druid_headers = {'Content-Type': 'application/json'}

if 'DRUID_HOST' not in os.environ.keys():
    druid_host=f"http://localhost:8888"
else:
    druid_host=f"http://{os.environ['DRUID_HOST']}:8888"

print(f"Opening a connection to {druid_host}.")
druid = druidapi.jupyter_client(druid_host)
display = druid.display
sql_client = druid.sql
status_client = druid.status

status_client.version

### Import additional modules

Run the following cell to import additional Python modules that you will use to call Druid APIs directly.

In [ ]:
import requests
import json

druid_headers = { 'Content-Type': 'application/json' }

## Create a table using batch ingestion

Run the following cell to create a table using batch ingestion. Specific dimensions are selected from the source data.

When completed, you'll see a description of the final table.

In [ ]:
table_name = 'example-wikipedia-compaction'

sql='''
REPLACE INTO "''' + table_name + '''" OVERWRITE ALL
WITH "ext" AS (
  SELECT *
  FROM TABLE(
    EXTERN(
      '{"type":"http","uris":["https://druid.apache.org/data/wikipedia.json.gz"]}',
      '{"type":"json"}'
    )
  ) EXTEND ("isRobot" VARCHAR, "channel" VARCHAR, "timestamp" VARCHAR, "flags" VARCHAR, "isUnpatrolled" VARCHAR, "page" VARCHAR, "diffUrl" VARCHAR, "added" BIGINT, "comment" VARCHAR, "commentLength" BIGINT, "isNew" VARCHAR, "isMinor" VARCHAR, "delta" BIGINT, "isAnonymous" VARCHAR, "user" VARCHAR, "deltaBucket" BIGINT, "deleted" BIGINT, "namespace" VARCHAR, "cityName" VARCHAR, "countryName" VARCHAR, "regionIsoCode" VARCHAR, "metroCode" BIGINT, "countryIsoCode" VARCHAR, "regionName" VARCHAR)
)
SELECT
  TIME_PARSE("timestamp") AS "__time",
  "namespace",
  "page",
  "user",
  "channel",
  "added",
  "deleted",
  "commentLength",
  "isRobot",
  "isAnonymous",
  "regionIsoCode",
  "countryIsoCode"
FROM "ext"
PARTITIONED BY HOUR
'''

display.run_task(sql)
sql_client.wait_until_ready(f'{table_name}')
display.table(f'{table_name}')

## Apply changes to data using compaction

Compaction is a special type of native [Druid task](https://druid.apache.org/docs/latest/ingestion/tasks#all-task-types) that, like streaming ingestion, uses JSON specifications to define behaviors. Each contains:

* An [ioConfig](https://druid.apache.org/docs/latest/data-management/manual-compaction#compaction-io-configuration), defining what the source data is for the job.
* A [tuningConfig](https://druid.apache.org/docs/latest/ingestion/native-batch#tuningconfig), detailing specific controls.
* And elements to control what happens to the data (as you would find in a [`dataSchema`](https://druid.apache.org/docs/latest/ingestion/ingestion-spec#dataschema) in streaming ingestion).
  * The dimensions to put into the resulting data given in a `dimensionsSpec`.
  * Any filters or calculations to do on the data as listed in the `transformsSpec`.
  * Any aggregation that should be done, as given in the `metricsSpec` when `rollup` is enabled.
 
In the cells that follow you will see various examples of how to use the `dimensionsSpec` and `transformsSpec` to affect table data as part of compaction.

### Use dimensionsSpec to add or remove columns

Amend the dimensions in the table at compaction time by using a [`dimensionsSpec`](https://druid.apache.org/docs/latest/data-management/manual-compaction#compaction-dimensions-spec). Options include removing (`dimensionExclusions`) or explicitly including (`dimensions`) specific columns.

As each segment defines its own schema, use the SYS.SEGMENTS table to view the dimensions of the table in each partition by running the cell below.

In [ ]:
sql=f'''
SELECT DISTINCT
    "start",
    "end",
    "dimensions"
FROM sys.segments
WHERE datasource = '{table_name}'
ORDER BY 1
'''

display.sql(sql)

The cell below will construct a compaction task specification that includes a [`dimensionsSpec`](https://druid.apache.org/docs/latest/data-management/manual-compaction#compaction-dimensions-spec) to remove specific columns.

* The `ioConfig` is represented by `compaction_ioConfig`. It contains an `inputSpec` that has a restriction on the `interval` so that this task only affects data between 19:00 and 20:00.
* The `granularitySpec` matches the original PARTITIONED BY.
* The `dimensionsSpec` contains an explicit list of `dimensionExclusions` - these are what will be removed.

Finally, the `compaction_spec` object uses these objects to create the final JSON compaction specification.

Run the cell to print out the JSON.

In [ ]:
compaction_ioConfig_inputSpec = {
    "type" : "interval",
    "interval" : "2016-06-27T19:00:00/PT1H" }

compaction_ioConfig = {
    "type" : "compact",
    "inputSpec" : compaction_ioConfig_inputSpec }

compaction_granularitySpec = { "segmentGranularity" : "HOUR" }

compaction_dimensionsSpec = {
    "dimensionExclusions" : [ "namespace", "isAnonymous", "user" ] }

compaction_spec = {
    "type": "compact",
    "dataSource": table_name,
    "ioConfig": compaction_ioConfig,
    "granularitySpec": compaction_granularitySpec,
    "dimensionsSpec": compaction_dimensionsSpec
}

print(json.dumps(compaction_spec, indent=2))

In [ ]:
requests.post(f"{druid_host}/druid/indexer/v1/task", json.dumps(compaction_spec), headers=druid_headers)

Run this cell below to follow along as the compaction task runs.

In [ ]:
sql=f'''
SELECT DISTINCT
    "start",
    "end",
    "dimensions"
FROM sys.segments
WHERE datasource = '{table_name}'
ORDER BY 1
'''

display.sql(sql)

When completed, you will see that the data between 1900 and 2000 no longer contains all the dimensions.

### Use a transformsSpec to filter out data

Incorporate a [native filter](https://druid.apache.org/docs/latest/querying/filters) into a `transformsSpec` during compaction to retain or remove rows that match a particular condition.

Run the following cell to retrieve some data from the table.

In [ ]:
sql=f'''
SELECT TIME_FLOOR("__time",'PT1H'),
   channel,
   COUNT(*) AS "events"
FROM "{table_name}"
WHERE TIME_IN_INTERVAL ("__time",'2016-06-27T00:00/P1D')
AND "channel" = '#en.wikipedia'
GROUP BY 1, 2
'''

display.sql(sql)

Run the cell below to construct a compaction specification that only retains events where the `channel` is `#en.wikipedia`. Notice that this change will only affect events in the table between 1000 and 1100.

In [ ]:
compaction_ioConfig_inputSpec = {
    "type" : "interval",
    "interval" : "2016-06-27T10:00:00/PT1H" }

compaction_ioConfig = {
    "type" : "compact",
    "inputSpec" : compaction_ioConfig_inputSpec }

compaction_granularitySpec = { "segmentGranularity" : "HOUR" }

compaction_transformSpec = {
    "filter":
        {
            "type": "selector",
            "dimension": "channel",
            "value": "#en.wikipedia"
        }
    }

compaction_spec = {
    "type": "compact",
    "dataSource": table_name,
    "ioConfig": compaction_ioConfig,
    "granularitySpec": compaction_granularitySpec,
    "transformSpec": compaction_transformSpec
}

print(json.dumps(compaction_spec, indent=2))

Run the next cell to submit the compaction.

In [ ]:
requests.post(f"{druid_host}/druid/indexer/v1/task", json.dumps(compaction_spec), headers=druid_headers)

Run the cell below to see how this has affected the data.

In [ ]:
sql=f'''
SELECT TIME_FLOOR("__time",'PT1H'),
   channel,
   COUNT(*) AS "events"
FROM "{table_name}"
WHERE TIME_IN_INTERVAL ("__time",'2016-06-27T10:00/PT1H')
GROUP BY 1, 2
'''

display.sql(sql)

When completed, the time period selected will _only_ contain rows for a particular channel.

In [ ]:
## Apply a GROUP BY (rollup) through compaction

Use metricsSpec and https://druid.apache.org/docs/latest/data-management/manual-compaction/#compaction-granularity-spec / rollup.

## Clean up

Run the following cell to remove the XXX used in this notebook from the database.

In [ ]:
# Use this for batch ingested tables

druid.datasources.drop(f"{table_name}")

# Use this when doing streaming with the data generator

print(f"Stop streaming generator: [{requests.post(f'{datagen_host}/stop/{datagen_job}','')}]")
print(f'Pause streaming ingestion: [{requests.post(f"{druid_host}/druid/indexer/v1/supervisor/{datagen_topic}/suspend","")}]')

print(f'Shutting down running tasks ...')

tasks = druid.tasks.tasks(state='running', table=table_name)
while len(tasks)>0:
    for task in tasks:
        print(f"...stopping task [{task['id']}]")
        druid.tasks.shut_down_task(task['id'])
    tasks = druid.tasks.tasks(state='running', table=table_name)

print(f'Reset offsets for re-runnability: [{requests.post(f"{druid_host}/druid/indexer/v1/supervisor/{datagen_topic}/reset","")}]')
print(f'Terminate streaming ingestion: [{requests.post(f"{druid_host}/druid/indexer/v1/supervisor/{datagen_topic}/terminate","")}]')
print(f"Drop datasource: [{druid.datasources.drop(table_name)}]")

## Summary

* You learned this
* Remember this

## Learn more

* Try this out on your own data
* Solve for problem X that is't covered here
* Read docs pages
* Watch or read something cool from the community
* Do some exploratory stuff on your own

In [ ]:
# Here are some useful code elements that you can re-use.

# When just wanting to display some SQL results
sql = f'''SELECT * FROM "{table_name}" LIMIT 5'''
display.sql(sql)

# When ingesting data and wanting to describe the schema
display.run_task(sql)
sql_client.wait_until_ready('{table_name}')
display.table('{table_name}')

# When you want to show the native version of a SQL statement
print(json.dumps(json.loads(sql_client.explain_sql(sql)['PLAN']), indent=2))

# When you want a simple plot
df = pd.DataFrame(sql_client.sql(sql))
df.plot(x='x-axis', y='y-axis', marker='o')
plt.xticks(rotation=45, ha='right')
plt.gca().get_legend().remove()
plt.show()

# When you want to add some query context parameters
req = sql_client.sql_request(sql)
req.add_context("useApproximateTopN", "false")
resp = sql_client.sql_query(req)

# When you want to compare two different sets of results
df3 = df1.compare(df2, keep_equal=True)
df3

# When you want to see some messages from a Kafka topic
from kafka import KafkaConsumer

consumer = KafkaConsumer(bootstrap_servers=kafka_host)
consumer.subscribe(topics=datagen_topic)
count = 0
for message in consumer:
    count += 1
    if count == 5:
        break
    print ("%d:%d: v=%s" % (message.partition,
                            message.offset,
                            message.value))
consumer.unsubscribe()